# Минипроект по иссследованию данных и примениению Airflow

In [2]:
pip install vk-api

     -------------------------------------- 48.6/48.6 kB 111.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import vk_api
import random

In [2]:
ads = pd.read_csv('ads_data_121288.csv')

In [3]:
ads.head()

,date,time,event,platform,ad_id,ad_cost_type,ad_cost
0,2019-04-01,2019-04-01 0:00:48,view,web,121288,CPM,187.4
1,2019-04-01,2019-04-01 0:04:41,view,ios,121288,CPM,187.4
2,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,187.4
3,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,187.4
4,2019-04-01,2019-04-01 0:08:46,view,ios,121288,CPM,187.4


In [4]:
ads.event.nunique()  # Количество уникальных значений по event

2

In [5]:
ads.event.unique() # Уникальные начения по event

array(['view', 'click'], dtype=object)

In [6]:
ads.date.unique() # Только две даты

array(['2019-04-01', '2019-04-02'], dtype=object)

### Необходимые метрики: 
Количество показов, количество кликов, CTR(отношение кликов к показам), сумма потраченных денег

### Отчет по объявлению 121288 за 2 апреля
Траты: Х рублей (+Y%)

Показы: X (+Y%)

Клики: X (-Y%)

CTR: X (-Y%)   

In [7]:
ads[ads['event']=='view'].groupby(['date','ad_id']).count().reset_index()

,date,ad_id,time,event,platform,ad_cost_type,ad_cost
0,2019-04-01,121288,491,491,491,491,491
1,2019-04-02,121288,93,93,93,93,93


In [8]:
ads_views = ads[ads['event']=='view'].groupby(['date','ad_id']).count().reset_index()[['date','ad_id', 'event']]
#ads_views.columns ['date','ad_id', 'views']

In [11]:
ads_clicks = ads[ads['event']=='click'].groupby(['date','ad_id']).count().reset_index()[['date','ad_id', 'event']]
# ads_clicks.columns['date','ad_id', 'clicks']

In [12]:
ads_views

,date,ad_id,event
0,2019-04-01,121288,491
1,2019-04-02,121288,93


In [13]:
ads_clicks

,date,ad_id,event
0,2019-04-01,121288,17
1,2019-04-02,121288,6


In [14]:
# Объединяем таблицы по 'date'и 'ad_id'
ads_ctr = pd.merge(ads_views, ads_clicks, on = ['date', 'ad_id'])

In [15]:
ads_ctr

,date,ad_id,event_x,event_y
0,2019-04-01,121288,491,17
1,2019-04-02,121288,93,6


In [16]:
# CTR (отношение кликов к показам), добавляем новый столбец
ads_ctr['CTR'] = ads_ctr['event_y']/ads_ctr['event_x']*100


In [24]:
ads_ctr

,date,ad_id,event_x,event_y,CTR,money
0,2019-04-01,121288,491,17,3.462322,92.0134
1,2019-04-02,121288,93,6,6.451613,17.4282


In [ ]:
# Рассчитать сумму потраченных денег можно по следующей формуле:
# Значение из колонки ad_cost разделить на 1000 и умножить на количество показов объявления.


In [18]:
ads['ad_action_cost']=ads['ad_cost']/1000
ads

,date,time,event,platform,ad_id,ad_cost_type,ad_cost,ad_action_cost
0,2019-04-01,2019-04-01 0:00:48,view,web,121288,CPM,187.4,0.1874
1,2019-04-01,2019-04-01 0:04:41,view,ios,121288,CPM,187.4,0.1874
2,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,187.4,0.1874
3,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,187.4,0.1874
4,2019-04-01,2019-04-01 0:08:46,view,ios,121288,CPM,187.4,0.1874
...,...,...,...,...,...,...,...,...
602,2019-04-02,2019-04-01 22:39:16,click,android,121288,CPM,187.4,0.1874
603,2019-04-02,2019-04-01 22:40:29,click,android,121288,CPM,187.4,0.1874
604,2019-04-02,2019-04-01 22:40:31,click,web,121288,CPM,187.4,0.1874
605,2019-04-02,2019-04-01 22:44:14,click,ios,121288,CPM,187.4,0.1874


In [19]:
#  Стоимость действия
ads['ad_action_cost'].unique()[0]

0.1874

In [32]:
# Стоимость затрат: показы *стоимость действия
ads_ctr['money'] = ads_ctr['event_x']*ads['ad_action_cost'].unique()[0]

In [33]:
ads_ctr

,date,ad_id,event_x,event_y,CTR,money
0,2019-04-01,121288,491,17,3.462322,92.0134
1,2019-04-02,121288,93,6,6.451613,17.4282


In [37]:
#Метрики за 02.04.2019
money_0204=float (ads_ctr[ads_ctr['date']=='2019-04-02']['money'])
view_0204=float (ads_ctr[ads_ctr['date']=='2019-04-02']['event_x'])
clicks_0204=float (ads_ctr[ads_ctr['date']=='2019-04-02']['event_y'])
ctr_0204=float (ads_ctr[ads_ctr['date']=='2019-04-02']['CTR'])

In [39]:
#Метрики за 01.04.2019
money_0104=float (ads_ctr[ads_ctr['date']=='2019-04-01']['money'])
view_0104=float (ads_ctr[ads_ctr['date']=='2019-04-01']['event_x'])
clicks_0104=float (ads_ctr[ads_ctr['date']=='2019-04-01']['event_y'])
ctr_0104=float (ads_ctr[ads_ctr['date']=='2019-04-01']['CTR'])

In [44]:
# Разница в процентах
diff_money= round((money_0204 - money_0104)/money_0104*100)
diff_view= round ((view_0204 - view_0104)/view_0104*100)
diff_clicks= round((clicks_0204 - clicks_0104)/clicks_0104*100)
diff_ctr= round((ctr_0204 - ctr_0104)/ctr_0104*100)

In [50]:
massage_vk= f'''Отчет по объявлению 121288 за 2 апреля\n
Траты: {money_0204} ({diff_money}%)
Показы: {view_0204} ({diff_view}%)
Клики: {clicks_0204} ({diff_clicks}%)
CTR: {ctr_0204} ({diff_ctr}%)
'''

In [51]:
# Записываем наш отчёт в текстовый файл
with open (f'report_2019-04-02.txt', 'w') as file:
    file.write(massage_vk)
    

In [ ]:
app_token =''
chat_id = 1
my_id = 
vk_session = vk_api.VkApi(token=app_token)
vk = vk_session.get_api()

vk.massages.send(chat_id=chat_id, 
                random_id=random.randint(1,2, ** 31),
                massage=massage)
